# Provisioning workers with SLURMCluster

We have a function called `whats_my_number` that takes a number as an argument and prints out what host the function is run on.

`report_results` appends the output from the various results and prints a report.

`elapsed` is a way of timing the execution (we could have used `%time` in this notebook, but we'll run similar code in a plain python script for comparison).

In [ ]:
from dask_jobqueue import SLURMCluster
from dask.distributed import Client
from dask import delayed

import time
import socket

def whats_my_number(num):
    out ='The number on host ' + socket.gethostname() + ' is ' + str(num)
    time.sleep(2)
    return out

def report_results(results):
    out = 'Joining on host ' + socket.gethostname() + '\n'
    out += '\n'.join(results)
    return out

# How many seconds since start?
def elapsed(start):
    return str(time.time() - start) + ' seconds'

In [ ]:
# Create the task graph
# 8 delayed calls to what's my number,
# Assemble output report

data = [1, 2, 3, 4, 5, 6, 7, 8]
results = []
for num in data:
    out = delayed(whats_my_number)(num)
    results.append(out)

report = delayed(report_results)(results)

In [ ]:
# Visualize the task graph
report.visualize()

## Create a 'SLURM Cluster' of workers and view dashboard

Note that no workers get provisioned yet

In [ ]:
cluster = SLURMCluster(project='cc-debug',
                       cores=1,
                       memory="1000MB",
                       walltime='00:15:00')
client = Client(cluster)
cluster

## We can check out the job submission script that will get sent to SLURM to provision each worker (one job per worker)

In [ ]:
print(cluster.job_script())

## Provision a worker and check the SLURM queue

In [ ]:
cluster.scale(1)

In [ ]:
!squeue -u $USER

## Run our task graph on one worker

In [ ]:
time1 = time.time()
result = report.compute()
print('Result computed:', elapsed(time1))
print('\nOutput:')
print(result)

## Scale up to 4 workers and run again

In [ ]:
cluster.scale(4)

In [ ]:
!squeue -u $USER

In [ ]:
time1 = time.time()
result = report.compute()
print('Result computed:', elapsed(time1))
print('\nOutput:')
print(result)

## Scale up to 8 workers and run again

In [ ]:
cluster.scale(8)

In [ ]:
!squeue -u $USER

In [ ]:
time1 = time.time()
result = report.compute()
print('Result computed:', elapsed(time1))
print('\nOutput:')
print(result)

## Clean up the workers

In [ ]:
# Shut down workers

cluster.close()
client.close()

In [ ]:
!squeue -u $USER